In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/HPV/clean_HPV_texts_phi3_Misinformation spreader.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/HPV/clean_HPV_texts_Phi_fear.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/HPV/clean_HPV_texts_phi3_Anti-Vacciner.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/HPV/clean_HPV_texts_phi_religious_conspiracy.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/COVID-19/clean_COVID-19_texts_phi3_Misinformation spreader.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/COVID-19/clean_COVID-19_texts_phi3_Anti-Vacciner.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/COVID-19/clean_COVID-19_texts__phi3_fear.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/COVID-19/clean_COVID-19_texts_phi_religious_conspiracy (2).csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/INFLUENZA/clean_Influenza_texts_phi_fear_monger.csv
/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/PHI3/INFLUENZA/clean_Influenza_texts_phi3

In [2]:
#!pip install transformers accelerate datasets evaluate -q
!pip install huggingface_hub -q
!pip install --upgrade transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 91.7 MB/s eta 0:00:00:00:01:01


In [ ]:
os._exit(0)

## Misinformation Spreader

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Misinformation spreader.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama3_Misinformation spreader.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama3_Misinformation spreader.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2025-07-14 13:37:18.512654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752500238.873108      95 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752500238.971937      95 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Classifying clean_Influenza_texts_llama3_Misinformation spreader.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama3_Misinformation spreader.csv:   1%|          | 1/100 [00:02<03:31,  2.14s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama3_Misinformation spreader.csv:   2%|▏         | 2/100 [00:02<02:07,  1.30s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama3_Misinformation spreader.csv:   3%|▎         | 3/100 [00:03<01:39,  1.03s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VE


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.4967
  ✅ Precision: 0.2483
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3318


## Religious Conspiracy

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama_reilgious.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama_religious.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama_religious.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Classifying clean_Influenza_texts_llama_reilgious.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   1%|          | 1/100 [00:06<10:21,  6.28s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   2%|▏         | 2/100 [00:12<10:10,  6.23s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   3%|▎         | 3/100 [00:18<10:03,  6.22s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.5233
  ✅ Precision: 0.2617
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3435


## Fear Monger

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama_fear_monger.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama_fear_monger.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama_fear_monger.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2025-07-14 15:18:47.576162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752506327.953671      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752506328.062727      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Classifying clean_COVID-19_texts_llama_fear_monger.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts_llama_fear_monger.csv:   1%|          | 1/100 [00:02<03:51,  2.33s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts_llama_fear_monger.csv:   2%|▏         | 2/100 [00:03<02:16,  1.39s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts_llama_fear_monger.csv:   3%|▎         | 3/100 [00:03<01:44,  1.08s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.4967
  ✅ Precision: 0.2483
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3318


## Anti-Vacciner

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Anti-Vacciner.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama3_Anti-Vacciner.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama3_Anti-Vacciner.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Classifying clean_Influenza_texts_llama3_Anti-Vacciner.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama3_Anti-Vacciner.csv:   1%|          | 1/100 [00:06<10:09,  6.16s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama3_Anti-Vacciner.csv:   2%|▏         | 2/100 [00:12<10:04,  6.17s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama3_Anti-Vacciner.csv:   3%|▎         | 3/100 [00:18<09:58,  6.17s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classify


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.5167
  ✅ Precision: 0.2583
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3407


## COVID-19

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama_religious.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama3_Misinformation spreader.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama3_Anti-Vacciner.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/COVID-19/clean_COVID-19_texts_llama_fear_monger.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2025-07-14 17:45:15.119124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752515115.333873      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752515115.394236      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Classifying clean_COVID-19_texts_llama_religious.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts_llama_religious.csv:   1%|          | 1/100 [00:01<02:43,  1.65s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts_llama_religious.csv:   2%|▏         | 2/100 [00:02<01:48,  1.11s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts_llama_religious.csv:   3%|▎         | 3/100 [00:03<01:30,  1.08it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_COVID-19_texts


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.4900
  ✅ Precision: 0.2450
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3289


## HPV

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama_religious.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama_fear_monger.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama3_Misinformation spreader.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/HPV/clean_HPV_texts_llama3_Anti-Vacciner.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2025-07-14 18:58:53.043632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752519533.231024      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752519533.286162      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Classifying clean_HPV_texts_llama_religious.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_HPV_texts_llama_religious.csv:   1%|          | 1/100 [00:01<02:39,  1.61s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_HPV_texts_llama_religious.csv:   2%|▏         | 2/100 [00:02<01:46,  1.09s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_HPV_texts_llama_religious.csv:   3%|▎         | 3/100 [00:03<01:28,  1.09it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_HPV_texts_llama_religious.csv:   4


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.4950
  ✅ Precision: 0.2475
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3311


## INFLUENZA

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama_reilgious.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Misinformation spreader.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Anti-Vacciner.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama_fear_monger.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2025-07-15 05:11:10.994354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752556271.318953      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752556271.412750      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Classifying clean_Influenza_texts_llama_reilgious.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   1%|          | 1/100 [00:02<03:37,  2.20s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   2%|▏         | 2/100 [00:02<02:10,  1.33s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   3%|▎         | 3/100 [00:03<01:41,  1.05s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.4975
  ✅ Precision: 0.2487
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3322


## Overall

In [ ]:
# === Imports ===
import os
import pandas as pd
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# === Login to Hugging Face ===
login("")  # <-- Replace with your real Hugging Face token

# === Load Mistral-7B-v0.3 ===
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# === Prompt formatting ===
def format_prompt(text):
    return f"""### Instruction:
Classify the following text as either Neutral or Misinformation.

### Text:
{text}

### Response:"""

# === Inference function ===
def classify(text):
    prompt = format_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Misinformation" in decoded:
        return "Misinformation"
    elif "Neutral" in decoded:
        return "Neutral"
    else:
        return "Unknown"

# === List your CSV files ===
csv_files = [
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama_reilgious.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Misinformation spreader.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama3_Anti-Vacciner.csv',
    '/kaggle/input/vaxguard/VaxGaurd-Dataset-roles/Llama 3/INFLUENZA/clean_Influenza_texts_llama_fear_monger.csv'
]

# === Collect predictions from all files ===
all_y_true = []
all_y_pred = []

for file_path in csv_files:
    df = pd.read_csv(file_path)
    df = pd.melt(df, var_name='label', value_name='text')
    df.dropna(inplace=True)
    df = df.sample(frac=1).reset_index(drop=True)
    df = df.head(100)  # Optional cap for faster processing

    y_true = df['label'].tolist()
    y_pred = []

    for text in tqdm(df['text'], desc=f"Classifying {os.path.basename(file_path)}"):
        y_pred.append(classify(text))

    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

# === Overall Evaluation Metrics ===
print("\n📊 === Overall Evaluation Across All Files ===")
overall_acc = accuracy_score(all_y_true, all_y_pred)
overall_prec = precision_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_rec = recall_score(all_y_true, all_y_pred, average="macro", zero_division=0)
overall_f1 = f1_score(all_y_true, all_y_pred, average="macro", zero_division=0)

print(f"  ✅ Accuracy:  {overall_acc:.4f}")
print(f"  ✅ Precision: {overall_prec:.4f}")
print(f"  ✅ Recall:    {overall_rec:.4f}")
print(f"  ✅ F1 Score:  {overall_f1:.4f}")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Classifying clean_Influenza_texts_llama_reilgious.csv:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   1%|          | 1/100 [00:06<10:19,  6.26s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   2%|▏         | 2/100 [00:12<10:08,  6.21s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_texts_llama_reilgious.csv:   3%|▎         | 3/100 [00:18<09:59,  6.18s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Classifying clean_Influenza_


📊 === Overall Evaluation Across All Files ===
  ✅ Accuracy:  0.5075
  ✅ Precision: 0.2537
  ✅ Recall:    0.5000
  ✅ F1 Score:  0.3367
